# Data Aggregation and Group Operations

## GroupBy Mechanics

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.101777,0.944597
1,a,two,1.076494,-0.085514
2,b,one,1.042342,0.584630
3,b,two,1.442695,-0.534952
4,a,one,1.677688,-0.076916


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a    0.884135
b    1.242518
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()

In [8]:
means.unstack()

key2,one,two
key1,,
a,0.787956,1.076494
b,1.042342,1.442695


In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [10]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [11]:
df['data1'].groupby([states,years]).mean().unstack()

,2005,2006
California,1.076494,1.042342
Ohio,0.670459,1.677688


In [12]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.787956  0.433841
     two   1.076494 -0.085514
b    one   1.042342  0.584630
     two   1.442695 -0.534952

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.884135,0.260722
b,1.242518,0.024839


In [14]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [15]:
for name, group in df.groupby('key1'):
    print(name) 
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.101777  0.944597
1    a  two  1.076494 -0.085514
4    a  one  1.677688 -0.076916
b
  key1 key2     data1     data2
2    b  one  1.042342  0.584630
3    b  two  1.442695 -0.534952


In [16]:
for(k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.101777  0.944597
4    a  one  1.677688 -0.076916
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.076494 -0.085514
('b', 'one')
  key1 key2     data1    data2
2    b  one  1.042342  0.58463
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.442695 -0.534952


In [17]:
pieces = dict(list(df.groupby('key1')))

In [18]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.042342,0.584630
3,b,two,1.442695,-0.534952


In [19]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [20]:
grouped = df.groupby(df.dtypes,axis=1)

In [21]:
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.101777  0.944597
1  1.076494 -0.085514
2  1.042342  0.584630
3  1.442695 -0.534952
4  1.677688 -0.076916
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [22]:
df.groupby('key1')['data1']
df.groupby('key1')['data2']

In [23]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.433841
     two  -0.085514
b    one   0.584630
     two  -0.534952

In [24]:
s_grouped = df.groupby(['key1','key2'])['data2']

In [25]:
s_grouped

In [26]:
s_grouped.mean()

key1  key2
a     one     0.433841
      two    -0.085514
b     one     0.584630
      two    -0.534952
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [27]:
people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [28]:
people.iloc[2:3,[1,2]] = np.nan

In [29]:
people

,a,b,c,d,e
Joe,-0.686373,-0.687011,-0.130912,-0.883987,-0.173930
Steve,-0.100750,-0.839414,-0.110769,1.202748,1.129331
Wes,0.415583,NaN,NaN,0.906101,1.134539
Jim,-1.617106,0.601392,1.176745,-1.413773,-2.073384
Travis,-1.828960,0.287875,1.001911,-1.446730,1.720943


In [30]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [31]:
by_column = people.groupby(mapping,axis=1)

In [32]:
by_column.sum()

,blue,red
Joe,-1.014899,-1.547314
Steve,1.091979,0.189168
Wes,0.906101,1.550122
Jim,-0.237028,-3.089099
Travis,-0.444819,0.179858


In [33]:
map_series = pd.Series(mapping)

In [34]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [35]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [36]:
people.groupby(len,axis=1).sum()

,1
Joe,-2.562212
Steve,1.281147
Wes,2.456224
Jim,-3.326127
Travis,-0.264960


In [37]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [38]:
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.686373 -0.687011 -0.130912 -0.883987 -0.173930
  two -1.617106  0.601392  1.176745 -1.413773 -2.073384
5 one -0.100750 -0.839414 -0.110769  1.202748  1.129331
6 two -1.828960  0.287875  1.001911 -1.446730  1.720943

### Grouping by Index Levels

In [39]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

In [40]:
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [41]:
hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)

In [42]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.310555  1.638321 -1.095317  1.275589 -0.420932
1      1.601476  0.641366 -0.746373 -0.871810 -0.451106
2      0.585531 -0.108422 -0.848185 -0.135701 -0.162038
3     -0.201436  1.446396 -0.204958 -0.003662  1.181144

In [43]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

In [44]:
df

,key1,key2,data1,data2
0,a,one,-0.101777,0.944597
1,a,two,1.076494,-0.085514
2,b,one,1.042342,0.584630
3,b,two,1.442695,-0.534952
4,a,one,1.677688,-0.076916


In [45]:
grouped = df.groupby('key1')

In [46]:
grouped['data1'].quantile(0.9)

key1
a    1.557449
b    1.402660
Name: data1, dtype: float64

In [47]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [48]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.779464,1.030111
b,0.400353,1.119581


In [49]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.884135  0.905193 -0.101777  0.487359  1.076494  1.377091   
b      2.0  1.242518  0.283092  1.042342  1.142430  1.242518  1.342607   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.677688   3.0  0.260722  0.592268 -0.085514 -0.081215 -0.076916   
b     1.442695   2.0  0.024839  0.791663 -0.534952 -0.255056  0.024839   

                          
           75%       max  
key1                      
a     0.433841  0.944597  
b     0.304734  0.584630

### Column-Wise and Multiple Function Application

In [50]:
tips = pd.read_csv(r"C:\Users\pengr\Desktop\pydata-book-master_ch08_tips.csv")

In [51]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

In [52]:
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [53]:
grouped = tips.groupby(['day','smoker'])

In [54]:
grouped_pct = grouped['tip_pct']

In [55]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [57]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [58]:
grouped_pct.agg([('foo','mean'),('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [59]:
functions = ['count','mean','max']

In [60]:
result = grouped['tip_pct','total_bill'].agg(functions)

In [61]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [62]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [63]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [64]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

### Returning Aggregated Data Without Row Indexes

In [68]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply:General split-apply-combine

In [69]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [70]:
top(tips,n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [73]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [74]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [81]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [82]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [83]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Suppressing the Group Keys

In [87]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [88]:
frame = pd.DataFrame({'data1': np.random.randn(1000),'data2': np.random.randn(1000)})

In [91]:
quartiles = pd.cut(frame.data1,4)

In [94]:
quartiles[:10]

0        (1.387, 2.82]
1    (-1.481, -0.0469]
2     (-0.0469, 1.387]
3     (-0.0469, 1.387]
4     (-0.0469, 1.387]
5     (-0.0469, 1.387]
6    (-1.481, -0.0469]
7    (-1.481, -0.0469]
8     (-0.0469, 1.387]
9     (-0.0469, 1.387]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.92, -1.481] < (-1.481, -0.0469] < (-0.0469, 1.387] < (1.387, 2.82]]

In [96]:
def get_stats(group):
    return{'min': group.min(), 'max': group.max(),'count': group.count(), 'mean': group.mean()}

In [97]:
grouped = frame.data2.groupby(quartiles)

In [100]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.92, -1.481]",74.0,2.387690,-0.061830,-2.401431
"(-1.481, -0.0469]",376.0,3.457215,0.080886,-2.662902
"(-0.0469, 1.387]",473.0,3.197369,-0.022389,-2.883950
"(1.387, 2.82]",77.0,2.329716,0.021075,-2.061430


In [101]:
grouping = pd.qcut(frame.data1,10,labels=False)

In [102]:
grouped = frame.data2.groupby(grouping)

In [103]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.552562,0.009657,-2.401431
1,100.0,3.457215,0.268232,-1.616877
2,100.0,3.081816,-0.102460,-2.015148
3,100.0,2.713358,0.072627,-2.662902
4,100.0,2.595812,0.037853,-2.529626
5,100.0,3.197369,-0.123626,-2.366535
6,100.0,3.009832,-0.033242,-2.663011
7,100.0,2.023822,-0.045485,-2.700152
8,100.0,2.549063,0.072688,-2.883950


### Example:Filling Missing Value with Group-Specific Values

In [104]:
s = pd.Series(np.random.randn(6))

In [105]:
s[::2] = np.nan

In [106]:
s

0         NaN
1   -0.639485
2         NaN
3   -0.077287
4         NaN
5    0.262114
dtype: float64

In [107]:
s.fillna(s.mean())

0   -0.151553
1   -0.639485
2   -0.151553
3   -0.077287
4   -0.151553
5    0.262114
dtype: float64

In [108]:
states = ['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']

In [109]:
group_key = ['East']*4 +['West']*4

In [110]:
data = pd.Series(np.random.randn(8),index=states)

In [111]:
data

Ohio          0.402197
New York      0.632918
Vermont       1.469741
Florida       0.576024
Oregon        0.487265
Nevada        1.982004
California    0.557991
Idaho        -0.138317
dtype: float64

In [112]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [113]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [115]:
data

Ohio          0.402197
New York      0.632918
Vermont            NaN
Florida       0.576024
Oregon        0.487265
Nevada             NaN
California    0.557991
Idaho              NaN
dtype: float64

In [114]:
data.groupby(group_key).mean()

East    0.537046
West    0.522628
dtype: float64

In [120]:
fill_mean = lambda g: g.fillna(g.mean())

In [121]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.402197
New York      0.632918
Vermont       0.537046
Florida       0.576024
Oregon        0.487265
Nevada        0.522628
California    0.557991
Idaho         0.522628
dtype: float64

In [122]:
fill_values = {'East':0.5,'West':-1}

In [123]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [124]:
data.groupby(group_key).apply(fill_func)

Ohio          0.402197
New York      0.632918
Vermont       0.500000
Florida       0.576024
Oregon        0.487265
Nevada       -1.000000
California    0.557991
Idaho        -1.000000
dtype: float64

### Random Sampling and Permutation

In [126]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [127]:
cards

['AH',
 '2H',
 '3H',
 '4H',
 '5H',
 '6H',
 '7H',
 '8H',
 '9H',
 '10H',
 'JH',
 'KH',
 'QH',
 'AS',
 '2S',
 '3S',
 '4S',
 '5S',
 '6S',
 '7S',
 '8S',
 '9S',
 '10S',
 'JS',
 'KS',
 'QS',
 'AC',
 '2C',
 '3C',
 '4C',
 '5C',
 '6C',
 '7C',
 '8C',
 '9C',
 '10C',
 'JC',
 'KC',
 'QC',
 'AD',
 '2D',
 '3D',
 '4D',
 '5D',
 '6D',
 '7D',
 '8D',
 '9D',
 '10D',
 'JD',
 'KD',
 'QD']

In [128]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [129]:
def draw(deck,n=5):
    return deck.sample(n)

In [130]:
draw(deck)

3H      3
7C      7
4H      4
JC     10
10D    10
dtype: int64

In [131]:
get_suit = lambda card:card[-1]

In [132]:
deck.groupby(get_suit).apply(draw,n=2)

C  6C     6
   4C     4
D  5D     5
   KD    10
H  2H     2
   6H     6
S  8S     8
   AS     1
dtype: int64

In [133]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

4C     4
7C     7
6D     6
2D     2
7H     7
2H     2
QS    10
8S     8
dtype: int64

### Example:Group Weighted Average and Correlation

In [135]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'], 
                   'data': np.random.randn(8), 'weights': np.random.rand(8)})

In [136]:
df

,category,data,weights
0,a,-0.504000,0.210267
1,a,1.349153,0.178316
2,a,1.965683,0.945877
3,a,-0.319697,0.504550
4,b,-0.706959,0.779116
5,b,-1.336998,0.565126
6,b,1.639157,0.901709
7,b,1.004775,0.036233


In [137]:
grouped = df.groupby('category')

In [138]:
get_wavg = lambda g: np.average(g['data'],weights=g['weights'])

In [139]:
grouped.apply(get_wavg)

category
a    0.996510
b    0.091173
dtype: float64

In [140]:
close_px = pd.read_csv(r"C:\Users\pengr\Desktop\pydata-book-master_ch09_stock_px.csv", parse_dates=True,index_col=0)

In [141]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [142]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [143]:
spx_corr = lambda x:x.corrwith(x['SPX'])

In [144]:
rets = close_px.pct_change().dropna()

In [145]:
get_year = lambda x:x.year

In [146]:
by_year = rets.groupby(get_year)

In [147]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [150]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [151]:
import statsmodels.api as sm

In [162]:
def regress(data,yvar,xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y,X).fit()
    return result.params

In [163]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation

In [164]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [168]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [169]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [170]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulation: Crosstab

In [172]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244